In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the Excel file
file_path = '/content/IRENA_Stats_extract_2024 H2.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the data from the "All data" sheet
df = excel_data.parse('All data')

# Filter data for Singapore
singapore_data = df[df['Country'] == 'Singapore']

# Further filter for renewable energy
renewable_singapore_data = singapore_data[singapore_data['RE or Non-RE'] == 'Total Renewable']

# Select relevant columns for plotting (Year, Installed Capacity and Electricity Generation)
renewable_singapore_data = renewable_singapore_data[['Year', 'Electricity Generation (GWh)', 'Electricity Installed Capacity (MW)']]

# Drop rows with missing data
renewable_singapore_data.dropna(subset=['Electricity Generation (GWh)', 'Electricity Installed Capacity (MW)'], inplace=True)

# Convert Installed Capacity (MW) to GWh using (Installed Capacity * 24 hours * 365 days) / 1000
renewable_singapore_data['Installed Capacity (GWh)'] = (renewable_singapore_data['Electricity Installed Capacity (MW)'] * 24 * 365) / 1000

# Group by year and sum the capacities and generation values
merged_renewable_data = renewable_singapore_data.groupby('Year').agg({
    'Electricity Generation (GWh)': 'sum',
    'Installed Capacity (GWh)': 'sum'
}).reset_index()

# Function to fit SARIMAX model and predict/forecast
def sarimax_model_predict(data, steps=4):
    # Train SARIMAX model
    model = SARIMAX(data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)

    # Get prediction for historical data
    predicted = model_fit.get_prediction(start=0, end=len(data)-1)
    predicted_values = predicted.predicted_mean

    # Forecast the next 'steps' years
    forecast = model_fit.get_forecast(steps=steps)
    forecast_values = forecast.predicted_mean

    # Extend index to account for forecast
    extended_index = list(data.index) + [data.index[-1] + i + 1 for i in range(steps)]

    # Create a DataFrame to hold results
    predicted_df = pd.DataFrame({
        'Actual': list(data) + [None] * steps,
        'Predicted': list(predicted_values) + [None] * steps,
        'Forecast': [None] * len(data) + list(forecast_values)
    }, index=extended_index)

    return predicted_df

# Apply SARIMAX for Electricity Generation (GWh)
generation_predicted_df = sarimax_model_predict(merged_renewable_data['Electricity Generation (GWh)'])

# Apply SARIMAX for Installed Capacity converted to GWh
capacity_predicted_df = sarimax_model_predict(merged_renewable_data['Installed Capacity (GWh)'])

# Plot Electricity Generation in GWh
plt.figure(figsize=(10, 6))
ax = plt.gca()

ax.plot(merged_renewable_data['Year'], generation_predicted_df['Actual'][:len(merged_renewable_data)], c='b', label='Actual Generation (GWh)')
ax.plot(merged_renewable_data['Year'], generation_predicted_df['Predicted'][:len(merged_renewable_data)], c='r', label='Predicted Generation (GWh)')
ax.plot(range(merged_renewable_data['Year'].iloc[-1]+1, merged_renewable_data['Year'].iloc[-1]+5), generation_predicted_df['Forecast'].iloc[-4:], c='g', label='Forecasted Generation (GWh)')

# Adding labels and title
plt.xlabel('Year')
plt.ylabel('Electricity Generation (GWh)')
plt.title('Electricity Generation Forecast (GWh)')
plt.legend()
plt.grid(True)
plt.show()

# Plot Installed Capacity in GWh
plt.figure(figsize=(10, 6))
ax = plt.gca()

ax.plot(merged_renewable_data['Year'], capacity_predicted_df['Actual'][:len(merged_renewable_data)], c='b', label='Actual Installed Capacity (GWh)')
ax.plot(merged_renewable_data['Year'], capacity_predicted_df['Predicted'][:len(merged_renewable_data)], c='r', label='Predicted Installed Capacity (GWh)')
ax.plot(range(merged_renewable_data['Year'].iloc[-1]+1, merged_renewable_data['Year'].iloc[-1]+5), capacity_predicted_df['Forecast'].iloc[-4:], c='g', label='Forecasted Installed Capacity (GWh)')

# Adding labels and title
plt.xlabel('Year')
plt.ylabel('Installed Capacity (GWh)')
plt.title('Installed Capacity Forecast (GWh)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
# Function to create Plotly line plot for generation or capacity
# Plot using Plotly for Electricity Generation
trace0b = go.Scatter(
    x = merged_renewable_data['Year'],
    y = generation_predicted_df['Actual'][:len(merged_renewable_data)],
    mode = 'lines',
    name = 'Actual Generation (GWh)',
    line=dict(color='blue')
)
trace1b = go.Scatter(
    x = merged_renewable_data['Year'],
    y = generation_predicted_df['Predicted'][:len(merged_renewable_data)],
    mode = 'lines',
    name = 'Predicted Generation (GWh)',
    line=dict(color='red')
)
trace2b = go.Scatter(
    x = list(range(merged_renewable_data['Year'].iloc[-1]+1, merged_renewable_data['Year'].iloc[-1]+5)),
    y = generation_predicted_df['Forecast'].iloc[-4:],
    mode = 'lines',
    name = 'Forecasted Generation (GWh)',
    line=dict(color='green')
)
data1 = [trace0b, trace1b, trace2b]

layout1 = go.Layout(
    title='Electricity Generation Forecast (GWh)',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Electricity Generation (GWh)')
)

fig1 = go.Figure(data=data1, layout=layout1)

# Plot using Plotly for Installed Capacity
trace0c = go.Scatter(
    x = merged_renewable_data['Year'],
    y = capacity_predicted_df['Actual'][:len(merged_renewable_data)],
    mode = 'lines',
    name = 'Actual Installed Capacity (GWh)',
    line=dict(color='blue')
)
trace1c = go.Scatter(
    x = merged_renewable_data['Year'],
    y = capacity_predicted_df['Predicted'][:len(merged_renewable_data)],
    mode = 'lines',
    name = 'Predicted Installed Capacity (GWh)',
    line=dict(color='red')
)
trace2c = go.Scatter(
    x = list(range(merged_renewable_data['Year'].iloc[-1]+1, merged_renewable_data['Year'].iloc[-1]+5)),
    y = capacity_predicted_df['Forecast'].iloc[-4:],
    mode = 'lines',
    name = 'Forecasted Installed Capacity (GWh)',
    line=dict(color='green')
)
data2 = [trace0c, trace1c, trace2c]

layout2 = go.Layout(
    title='Installed Capacity Forecast (GWh)',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Installed Capacity (GWh)')
)

fig2 = go.Figure(data=data2, layout=layout2)

# Export to an HTML file
pyo.plot(fig1, filename='electricity_generation_forecast.html')
pyo.plot(fig2, filename='installed_capacity_forecast.html')